In [1]:
import os
import gc
import cv2
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

# Utils
from tqdm import tqdm
from collections import defaultdict

# Model Import
from ResNet20 import resnet20

In [2]:
CONFIG = dict(
    seed = 42,
    valid_batch_size = 256,
    num_classes = 10,
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

In [3]:
MODEL_PATHS = 'ResNet20 final.bin'

In [4]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [5]:
validation_loader = torch.utils.data.DataLoader(
        torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])),
        batch_size=CONFIG['valid_batch_size'], shuffle=False,
        num_workers=2, pin_memory=True)

In [6]:
def criterion(outputs, labels):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, labels)

In [7]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    correct = 0.0
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader), ncols=100)
    for step, data in bar:        
        inputs, targets = data
        inputs = inputs.to(device)
        targets = targets.to(device)
            
        # рачсет вывода
        output = model(inputs)
        loss = criterion(output, targets)
        
        _, preds = output.max(1)
        correct += preds.eq(targets).sum()

        batch_size = inputs.size(0)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        sum_loss = running_loss / dataset_size
        
        sum_score = correct.cpu().detach().numpy() / dataset_size
        
        bar.set_postfix({'Valid_Loss':sum_loss, 'Valid_Score':sum_score})
        PREDS.append(output.view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    
    gc.collect()
    return sum_loss, sum_score, PREDS

In [8]:
def performance_inference(model_path, dataloader, device,):
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
        
    model = resnet20()
    model.to(CONFIG['device'])
    model.load_state_dict(torch.load(model_path))
    
    start = time.time()
    history = defaultdict(list)
    
    start = time.time()
        
    val_loss, val_score, preds = valid_fn(model, dataloader, device=CONFIG['device'])
    
    end = time.time()
    
    history['Valid Loss'].append(val_loss)
    history['Valid Score'].append(val_score)
    
    time_elapsed = end - start
    print('Validation complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Validation Loss: {:.4f}".format(val_loss))
    print("Validation Score: {:.4f}".format(val_score))
    
    
    return model, history

In [9]:
model, history = performance_inference(MODEL_PATHS, validation_loader, CONFIG['device'])

[INFO] Using GPU: NVIDIA GeForce RTX 2080



100%|██████████████████████████| 40/40 [00:00<00:00, 59.09it/s, Valid_Loss=0.279, Valid_Score=0.914]


Validation complete in 0h 0m 1s
Validation Loss: 0.2794
Validation Score: 0.9141
